In [6]:

import cassiopeia as cass
import sqlite3
import pandas as pd
import math
import time 
import random

def auto_retry(api_call_method):
    """ A decorator to automatically retry 500/503s (Service Unavailable) and skip 400s (Bad Request) or 404 (Not Found). """
    def call_wrapper(*args, **kwargs):
        try:
            return api_call_method(*args, **kwargs)
        except APIError as error:
            # try again once
            if error.error_code in [500, 503]:
                try:
                    print("Got a 500 or 503, trying again after 5 seconds...")
                    time.sleep(5)
                    return api_call_method(*args, **kwargs)
                except APIError as another_error:
                    if another_error.error_code in [500, 503, 400, 404]:
                        pass
                    else:
                        raise another_error

            # skip
            elif error.error_code in [400, 404]:
                print("Got a 400 or 404")
                pass # may make match None in auto_retry(riotapi.get_match)!

            # fatal
            else:
                raise error
    return call_wrapper




# How to scrape League of Legends Data?
### Introduction
- League of Legends is a moba game. The most basic game is between blue and red team. Each team has 5 "summoners" aka players.
- Rank in League of Legends: The highest rank in League is Challenger. Most professional players and top players practice in KR server (Korea). Many of them are challengers. 
- I want to build a classifer to predict the match restul. 

### Mission
- build a classifier to predict match result: which team will win the game? Based on some features of the match
    + 1. champion of both teams
    + 2. adding more info: first kill, first baron
    + 3. (To be continued)
### Preparation

- `pip install cassiopeia` to install cassiopeia
- go to RiotAPI website to create your own temporary api [https://developer.riotgames.com/]
- copy the api `key` from what you created
- try the following cell out! Replace the key, region and name to your own!

In [ ]:
import random

import cassiopeia as cass
your_key = "RGAPI-bde8646a-39e7-4933-9736-992db586ce1c"
your_region = "NA"
your_name = "DirectKill"
cass.set_riot_api_key(your_key)  # This overrides the value set in your configuration/settings.
cass.set_default_region(your_region)
summoner = cass.get_summoner(name=your_name)
print("{name} is a level {level} summoner on the {region} server.".format(name=summoner.name,
                                                                          level=summoner.level,
                                                                          region=summoner.region))



### Start from the top, challenger Number 1
- I want to build a classifier that can predict games in the challenger league in Korea Server. 
- I am starting from the number 1 in Challenger league. So first, we need to find the id of that person. 

In [190]:
#how to find the number one in default region (NA)
challenger_league = cass.get_challenger_league(queue=cass.Queue.ranked_solo_fives)
best_na = challenger_league[0].summoner
print("The number one in NA server right now is: {name} ".format(name=best_na.name))

He's not as good as nxi at League, but probably a better python programmer!


In [214]:
#now we want to find number one in Korea (KR)
challenger_league = cass.get_challenger_league(queue=cass.Queue.ranked_solo_fives,region="KR")
summoner = Summoner(name=challenger_league.entries[0].summoner.name, region="KR")
summoner.name

'군밤갓'

In [302]:
challenger_league = cass.get_challenger_league(queue=cass.Queue.ranked_solo_fives,region="KR")
points = challenger_league.entries[1].league_points


In [325]:
challenger_league = cass.get_challenger_league(queue=cass.Queue.ranked_solo_fives,region="NA")

summoner = Summoner(name=challenger_league.entries[3].summoner.name, region="NA")
summoner.champion_masteries.find("Pyke").points

148783

### BFS to find match ids
- Perform a BFS to search for 5 games for our KR number 1 summoner. 
- Append five matches he/she played to the match list. If the match id already in the match list, then skip and use the next one. 
- In these five games, randomly choose 1 match and append the other 9 summoners of this match to the Summoner queue. If summoner already in the queue or have been popped already, skip. 
- Pop next one from the Summoner queue, and continue finding 5 games played by this summoner. 
- Filter: the five games have to be ranked solo 5v5, and has to be patch 10.8
- I limit my BFS to 100 players. So there should be a total of 500 match id appended to the match list. 

In [224]:
import random
from sortedcontainers import SortedList
import arrow

from cassiopeia.core import Summoner, MatchHistory, Match
from cassiopeia import Queue, Patch, Season


def filter_match_history(summoner,patch):
    end_time = patch.end
    if end_time is None:
        end_time = arrow.now()
    match_history = MatchHistory(summoner=summoner, begin_time=patch.start, end_time=end_time, queues={Queue.ranked_solo_fives})
    return match_history


def collect_matches():
    initial_summoner_name = "군밤갓"
    region = "KR"

    summoner = Summoner(name=initial_summoner_name, region=region)
    

    unpulled_summoner_ids = SortedList([summoner.id])
    pulled_summoner_ids = SortedList()

    unpulled_match_ids = SortedList()
    pulled_match_ids = SortedList()
    n = 0
    while unpulled_summoner_ids and n< 100:
        # Get a random summoner from our list of unpulled summoners and pull their match history
        new_summoner_id = random.choice(unpulled_summoner_ids)
        
        new_summoner = Summoner(id=new_summoner_id, region=region)
        patch = Patch.from_str("10.8", region='NA')

        matches = filter_match_history(new_summoner,patch)
        unpulled_match_ids.update([match.id for match in matches])
        unpulled_summoner_ids.remove(new_summoner_id)
        pulled_summoner_ids.add(new_summoner_id)
        n = n+1
        x = 0
        while unpulled_match_ids and x <5:
            # Get a random match from our list of matches
            new_match_id = random.choice(unpulled_match_ids)
            
            new_match = Match(id=new_match_id, region=region)
            for participant in new_match.participants:
                if participant.summoner.id not in pulled_summoner_ids and participant.summoner.id not in unpulled_summoner_ids:
                    unpulled_summoner_ids.add(participant.summoner.id)
            # The above lines will trigger the match to load its data by iterating over all the participants.
            # If you have a database in your datapipeline, the match will automatically be stored in it.
            unpulled_match_ids.remove(new_match_id)
            
            pulled_match_ids.add(new_match_id)
            x = x+1
    return pulled_match_ids

if __name__ == "__main__":
    a = collect_matches()

Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/vq3db4tpenrE6hSk2Vt478GyX0j4NDyTiBA4ymxJKWse?beginIndex=0&endIndex=100&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/U9H6s4EAcqeQPrAjU867OqrG9jvD3Yeb0nDOi537tEP4kyg?beginIndex=0&endIndex=100&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/U9H6s4EAcqeQPrAjU867OqrG9jvD3Yeb0nDOi537tEP4kyg?beginIndex=100&endIndex=200&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4335302311
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4325407236
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4315541899
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4323256685
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4330923253
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/Yy73eJKLF2eSc0M4t32XQf05425PAHo5LYWQAAKBkDseo_C666J0yP7i?beginIndex=0&e

Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/xL93tGVSZKC1roEbU6jyg9G4Ymm5ElpP6lnOaQROuld5Klo?beginIndex=300&endIndex=400&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4283340070
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4320278576
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4323418731
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4315222884
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4323314876
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/0Hfc4G_BI-7W3dPZMnIR7DHUl3Pz-9drPClP7ghmf0Jz1nMZ5kdBPzx1?beginIndex=0&endIndex=100&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4333965745
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4290274882
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4299115129
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4280808126
M

Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4304415067
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4325684135
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/r7jnFVm1c754WHcBC-Pm57eZIE0_hOiHUsYHLJXAUMOv?beginIndex=0&endIndex=100&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4281732795
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4304261854
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4283208346
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4315879719
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4318313761
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/xRC6GufLzyNphcdiho1lpaU5W54siWROXcKup04o_DgD5slhOWZ7YPTj?beginIndex=0&endIndex=100&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/xRC6GufLzyNphcdiho1lpaU5W54siWROXcKup04o_DgD5slhOWZ7YPTj?beginIndex=100&endIn

Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4331538271
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4321024059
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4320166095
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4293940696
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/ixhdkSA7ngQwtH7gW_bcwp5RkVDzrSQUuSlIHXislImL?beginIndex=0&endIndex=100&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4309125259
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4280012407
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4286778722
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4315530698
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4295673605
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/vfa6BCH024roraXRXzLvHOxaYQiTS3v34q5vCFdyDfB2?beginIndex=0&endIndex=100&queue=420
Making call: https

Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4297762377
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4292793284
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/8ZFkIpxkZkzs7ALGteQCqomQUAzuMQRBtI_Cb2PE5MGW-YMwpD2ojG7q?beginIndex=0&endIndex=100&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4315248595
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4324716929
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4281455338
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4312091733
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4291471081
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/y7EdqOAjov9BNIBg7a-ZjEgrWFyiHYX-ssaPoWjbZ0JPvXw?beginIndex=0&endIndex=100&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/y7EdqOAjov9BNIBg7a-ZjEgrWFyiHYX-ssaPoWjbZ0JPvXw?beginIndex=100&endIndex=20

Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4299193012
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4290396155
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4326161649
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4330198430
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4327173102
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/gddz34ckj9XjUWJlx7XfqZxaV1KgHeINHMSJtiAoGVmX7DCotEI0VUH1?beginIndex=0&endIndex=100&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/gddz34ckj9XjUWJlx7XfqZxaV1KgHeINHMSJtiAoGVmX7DCotEI0VUH1?beginIndex=100&endIndex=200&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/gddz34ckj9XjUWJlx7XfqZxaV1KgHeINHMSJtiAoGVmX7DCotEI0VUH1?beginIndex=200&endIndex=300&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4290974097
Making call: https://kr.api.riotgames.com/

Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/apiG4LKyftSiMNw37sI5uhk0iURRdiYxps81T25iF2gNp0BA9sTNigtg?beginIndex=100&endIndex=200&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4315497778
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4307270768
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4303186969
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4289417444
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/zsxDikL_hEfGUyyVIiZVY48CWVudBEB1zAgBBS2cxm2TlAjB5fy41T1j?beginIndex=0&endIndex=100&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/zsxDikL_hEfGUyyVIiZVY48CWVudBEB1zAgBBS2cxm2TlAjB5fy41T1j?beginIndex=100&endIndex=200&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/zsxDikL_hEfGUyyVIiZVY48CWVudBEB1zAgBBS2cxm2TlAjB5fy41T1j?beginIndex=200&endIndex=300&queue=420
Making call: https

Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/ERIdjv0nBzi1GW-dV8o2QK7wTCxzMsDa2mRcfPlr24fLr94?beginIndex=0&endIndex=100&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/ERIdjv0nBzi1GW-dV8o2QK7wTCxzMsDa2mRcfPlr24fLr94?beginIndex=100&endIndex=200&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4315536255
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4296435657
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4328691320
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4293884883
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4328482948
Making call: https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/rcS0CymcuT_n142_W-657pT-IIUyThV2Zquo7AGFI3s5DNBetKLbdSV4?beginIndex=0&endIndex=100&queue=420
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/4318405317
Making call: https://kr.api.riotgames.com/lol/match/v4/matches

In [227]:
matches = SortedList([4279795226, 4280012407, 4280241499, 4280294839, 4280368614, 4280407414, 4280413909, 4280440612, 4280508950, 4280536849, 4280613082, 4280791605, 4280808126, 4280845541, 4281267101, 4281414212, 4281455119, 4281455338, 4281455779, 4281551160, 4281732795, 4281749551, 4281763429, 4281917221, 4281966557, 4282275305, 4282444046, 4282619875, 4282746164, 4282914242, 4282926399, 4282959226, 4282999017, 4283068171, 4283114688, 4283146082, 4283161998, 4283208346, 4283340070, 4283565545, 4283779618, 4283994328, 4284316845, 4284471411, 4284473561, 4284479738, 4284509478, 4284548327, 4284550768, 4284644213, 4284674115, 4284685667, 4284719910, 4284766324, 4284802519, 4284881522, 4284897631, 4284897885, 4284899754, 4285085376, 4285096617, 4285268595, 4285450188, 4285469534, 4285474679, 4285625628, 4285711529, 4285913664, 4285926011, 4285926011, 4285936223, 4286561709, 4286605585, 4286629576, 4286778722, 4286799134, 4286914920, 4287282849, 4287351519, 4287416767, 4287581409, 4287588721, 4287651109, 4287734161, 4287740799, 4287845812, 4287869969, 4287869969, 4288021761, 4288142518, 4288287549, 4288561707, 4288713317, 4289417444, 4289505489, 4289990613, 4290274882, 4290276514, 4290381728, 4290388837, 4290396155, 4290483544, 4290491440, 4290510553, 4290654618, 4290702160, 4290709886, 4290737944, 4290745473, 4290759886, 4290795097, 4290861032, 4290872761, 4290882785, 4290890353, 4290890353, 4290964644, 4290974097, 4291360339, 4291471081, 4291534237, 4291747894, 4292194811, 4292240925, 4292349470, 4292387582, 4292404385, 4292445454, 4292514753, 4292539218, 4292553354, 4292599065, 4292791347, 4292793284, 4292858645, 4292858645, 4292876706, 4293248436, 4293374121, 4293397477, 4293518826, 4293520551, 4293540988, 4293586823, 4293690528, 4293705254, 4293746628, 4293750288, 4293787442, 4293812161, 4293850923, 4293875221, 4293884883, 4293940696, 4294237599, 4294369197, 4294402006, 4294418933, 4294605526, 4294796450, 4294801505, 4294973641, 4295058155, 4295164917, 4295320010, 4295529244, 4295653166, 4295673605, 4296149225, 4296182991, 4296268549, 4296378040, 4296406593, 4296435657, 4296468046, 4296470223, 4296480295, 4296950215, 4297272228, 4297303368, 4297315453, 4297333777, 4297466500, 4297559367, 4297559367, 4297577559, 4297622995, 4297762377, 4298051127, 4298393684, 4298864381, 4298866767, 4298940434, 4298997187, 4299028649, 4299109363, 4299115129, 4299117487, 4299193012, 4299582062, 4299612035, 4299650000, 4300004047, 4300067873, 4300156530, 4300214335, 4300227903, 4300242090, 4300266595, 4300359869, 4300377745, 4300508675, 4300587457, 4300754752, 4300851469, 4300922370, 4301179434, 4301291660, 4301343928, 4301402045, 4301633892, 4301735422, 4302360843, 4302750809, 4303109824, 4303186969, 4303192020, 4303292181, 4304026132, 4304043311, 4304063184, 4304129555, 4304156044, 4304261854, 4304415067, 4304548238, 4304559053, 4304646591, 4304656369, 4304718724, 4304795743, 4304825856, 4304874029, 4304972035, 4305573825, 4306024160, 4306106101, 4306282322, 4306497911, 4306820168, 4307116542, 4307146998, 4307174183, 4307242487, 4307270768, 4307298131, 4307310365, 4307511902, 4307543296, 4308009171, 4308251141, 4308338798, 4308557181, 4308696625, 4308794965, 4309018547, 4309071915, 4309074656, 4309125259, 4309208734, 4309403407, 4309426868, 4309934035, 4309960782, 4309965360, 4310066464, 4310207863, 4310242824, 4310297988, 4310880081, 4310956519, 4311194593, 4311555784, 4311669099, 4311786852, 4311912413, 4311994044, 4312018457, 4312078612, 4312091733, 4312260682, 4312359452, 4312559608, 4312621783, 4312698385, 4312863789, 4312866082, 4313048563, 4313077853, 4313088595, 4313239977, 4313248101, 4313248101, 4313692610, 4313762410, 4313959307, 4314260032, 4314377718, 4314393759, 4314558996, 4314951750, 4314995922, 4315046464, 4315088248, 4315135761, 4315143601, 4315157116, 4315177739, 4315222884, 4315248595, 4315270125, 4315381633, 4315417184, 4315418875, 4315454160, 4315465754, 4315497778, 4315530698, 4315536255, 4315541899, 4315565894, 4315720110, 4315744491, 4315825166, 4315837460, 4315874670, 4315874670, 4315879719, 4315882669, 4315895401, 4315971901, 4316793809, 4316969995, 4317207977, 4317305353, 4317410958, 4317410958, 4317431396, 4317822177, 4317939964, 4317954261, 4318076332, 4318168714, 4318219572, 4318251054, 4318302409, 4318313761, 4318350594, 4318376464, 4318405317, 4318478683, 4318540167, 4318875232, 4319032191, 4319064347, 4319283828, 4319290721, 4319438946, 4319744087, 4319860556, 4319863009, 4319916557, 4320046129, 4320166095, 4320166930, 4320217183, 4320220179, 4320235203, 4320278576, 4320440840, 4320572108, 4320731349, 4320768592, 4320788797, 4320788797, 4320933508, 4320964939, 4320986470, 4321024059, 4321627561, 4321631824, 4321846653, 4321904983, 4322128941, 4322159974, 4322160314, 4322366303, 4322806951, 4322916712, 4323051091, 4323139828, 4323145481, 4323218420, 4323233217, 4323256685, 4323314876, 4323318339, 4323396660, 4323418731, 4323429827, 4323442169, 4323462685, 4323469875, 4323484700, 4323502413, 4324104448, 4324389113, 4324413962, 4324602593, 4324624889, 4324716929, 4324741173, 4324768997, 4324911183, 4325236236, 4325245525, 4325331294, 4325402965, 4325407236, 4325465944, 4325684135, 4325857968, 4326075339, 4326118747, 4326157540, 4326161649, 4326164550, 4326200563, 4326215945, 4326467063, 4326549780, 4326549780, 4326941401, 4327076900, 4327098448, 4327173102, 4327315913, 4327459621, 4327484469, 4327556080, 4328006904, 4328410787, 4328482948, 4328581823, 4328630580, 4328640061, 4328640118, 4328675748, 4328691320, 4328750154, 4328828156, 4328836356, 4329422460, 4330052749, 4330170858, 4330181425, 4330198430, 4330201073, 4330201073, 4330259900, 4330297628, 4330356628, 4330409802, 4330806832, 4330825469, 4330866362, 4330923253, 4331044457, 4331108767, 4331261674, 4331334427, 4331486013, 4331538271, 4331690647, 4331715427, 4331766849, 4332840632, 4333126739, 4333498150, 4333716388, 4333782154, 4333794456, 4333872519, 4333916833, 4333920550, 4333965745, 4335004021, 4335300776, 4335302311, 4335422897])


In [364]:
match = Match(id = matches[100],region="KR")
blue_team = match.blue_team
random_person = blue_team.participants[2]
random_person.rank_last_season

AttributeError: 'ParticipantData' object has no attribute 'rankLastSeason'

In [374]:
for i in red_team.participants:
    participant_timeline = i.timeline
    role = participant_timeline.role.value
    lane = participant_timeline.lane
    print(role,lane)

DUO None
DUO_SUPPORT None
DUO_SUPPORT None
DUO_SUPPORT None
DUO_SUPPORT None


In [367]:
match.creation

<Arrow [2020-04-10T14:28:37.990000+00:00]>

In [237]:
from cassiopeia.core import Champions
def get_champions():
    champions = Champions(region="KR")
    champion_name = []
    champion_id = []
    
    for champion in champions:
        champion_name.append(champion.name)
        champion_id.append(champion.id)
    return dict(zip(champion_name,champion_id))
champion_dict = get_champions()

In [375]:
a = [i+" blue" for i in champion_dict.keys()]
b = [i + " red" for i in champion_dict.keys()]
combined = a+b +["blue_win","blue_first_dragon","blue_first_baron"]
df = pd.DataFrame(columns= combined)


In [377]:
for i in matches[1:50]:
    row = dict((k,0) for k in df.columns)
    matchid = i
    match = Match(id = i,region="KR")
    red_team = match.red_team.participants
    blue_team = match.blue_team.participants
    row["blue_win"] = match.blue_team.win
    row["blue_first_dragon"] = match.blue_team.first_dragon
    row["blue_first_baron"] = match.blue_team.first_baron
#     row["blue_first_dragon"] = match.blue_team.first_dragon
    for k in blue_team:
        champion = k.champion.name
        row[champion+" blue"] = 1
    
    for k in red_team:
        champion = k.champion.name
        row[champion+" red"] = 1
        
    df = df.append(row,ignore_index = True)
df
        

,아트록스 blue,아리 blue,아칼리 blue,알리스타 blue,아무무 blue,애니비아 blue,애니 blue,아펠리오스 blue,애쉬 blue,아우렐리온 솔 blue,...,유미 red,자크 red,제드 red,직스 red,질리언 red,조이 red,자이라 red,blue_win,blue_first_dragon,blue_first_baron
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,False,False,False
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,True,True,True
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,True,False,False
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,True,True,True
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,True,True,False
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,True,True,False
6,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,True,True,False
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,False,False,False
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,False,True,False
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,True,False,True


In [383]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.drop("blue_win",axis=1), df["blue_win"], test_size=0.3, random_state=42)
y_train = y_train.astype('int')
y_test = y_test.astype('int')

In [265]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
from sklearn.ensemble import RandomForestClassifier


In [386]:
clif = RandomForestClassifier()
clif.fit(X_train,y_train)
clif.score(X_test,y_test)

C:\Users\Wendy W\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.5333333333333333

0.58